In [23]:
import sys
print (sys.version)

2.7.6 (default, Nov 23 2017, 15:49:48) 
[GCC 4.8.4]


In [24]:
import pandas as pd
import numpy as np
import re
import unidecode

from keras.models import Sequential
from keras.layers import LSTM, Dropout, Activation, Dense
import string
from keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow.python.client import device_lib
from keras import backend as K
K.tensorflow_backend._get_available_gpus()


['/job:localhost/replica:0/task:0/device:GPU:0']

In [25]:
df = pd.read_csv("../data/petrogustavo.csv" , encoding="utf-8" )

In [26]:
df.head()

,favorite_count,source,text,in_reply_to_screen_name,is_retweet,created_at,retweet_count,id_str
0,33,Twitter for Android,La bombero del Distrito Zulima Muñoz murió en ...,NaN,False,Thu Nov 27 14:50:53 +0000 2014,32,537981895733424130
1,363,Twitter Web Client,El progresismo será un movimiento independient...,NaN,False,Fri Nov 25 00:32:15 +0000 2016,183,801946567095779328
2,545,Twitter for Android,Esta es la verdadera causa de la bancarrota de...,NaN,False,Sun Nov 26 20:19:03 +0000 2017,374,934879192126287873
3,59,Twitter for Android,_ @FelippeDuke la experiencia del crédito fue ...,FelipeDuqueM1,False,Tue Feb 09 16:18:01 +0000 2016,43,697092093857165312
4,31,Twitter for Android,El magisterio es una fuerza laboral esencialme...,NaN,False,Mon Oct 05 15:01:19 +0000 2015,58,651049533921718272


In [27]:
df["text"] = df["text"].apply( lambda x :  x.lower() )
df["text"] = df["text"].apply(lambda x:  re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE))
df["text"] = df["text"].apply(lambda x:  re.sub(r'^http?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE))

df["text"] = df["text"].apply(lambda x:   unidecode.unidecode(x )   )


In [28]:
characters = list(string.printable)

In [29]:
characters = list(string.printable)
characters.remove('\x0b')
characters.remove('\x0c')

In [30]:
VOCABULARY_SIZE = len(characters)
characters_to_ix = {c:i for i,c in enumerate(characters)}
print("vocabulary len = %d" % VOCABULARY_SIZE)
print(characters)

vocabulary len = 98
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' ', '\t', '\n', '\r']


In [31]:
N_GPU = 1 # you can experiment with more GPUs, it gets interesting with a high SEQUENCE_LEN
SEQUENCE_LEN = 100
BATCH_SIZE = 64
EPOCHS = 20
HIDDEN_LAYERS_DIM = 256
LAYER_COUNT = 4
DROPOUT = 0.2

In [32]:
def build_model( layer_count , sequence_len , vocab_size , dropout ):
    hidden_layers_dim = HIDDEN_LAYERS_DIM
    model = Sequential()
    for i in range( layer_count ):
        
        model.add(
            LSTM(
                hidden_layers_dim , 
                return_sequences=True if (i!=(layer_count-1)) else False,
                input_shape=( sequence_len , vocab_size ),
            )
        )
        model.add( Dropout(dropout))
        
    model.add(Dense(VOCABULARY_SIZE))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer="adam")
    return model

In [33]:
def describe_batch(X, y, samples=3):
    """Describe in a human-readable format some samples from a batch"""
    for i in range(samples):
        sentence = ""
        for s in range(SEQUENCE_LEN):
            sentence += characters[X[i,s,:].argmax()]
        next_char = characters[y[i,:].argmax()]
        
        print("sample #%d: ...%s -> '%s'" % (
            i,
            sentence[-20:],
            next_char
        ))

def batch_generator(text, count):
    """Generate batches for training"""
    while True: # keras wants that for reasons
        for batch_ix in range(count):
            X = np.zeros((BATCH_SIZE, SEQUENCE_LEN, VOCABULARY_SIZE))
            y = np.zeros((BATCH_SIZE, VOCABULARY_SIZE))

            batch_offset = BATCH_SIZE * batch_ix

            for sample_ix in range(BATCH_SIZE):
                sample_start = batch_offset + sample_ix
                for s in range(SEQUENCE_LEN):
                    X[sample_ix, s, characters_to_ix[text[sample_start+s]]] = 1
                y[sample_ix, characters_to_ix[text[sample_start+s+1]]]=1

            yield X, y

In [34]:
df_full = df["text"].sample(frac=1).reset_index(drop=True)


In [35]:
df_full.shape

(14234,)

In [36]:

df_train = df_full[:12000]
df_test = df_full[12000:]


In [37]:
#  \\ I will use this indicator of the end of a twitt  
full_train_text = "\\".join( df_train[:] )
full_test_text = "\\".join( df_test[:] )

In [38]:
train_len = len( full_train_text) 

In [39]:
test_len = len( full_test_text )

In [40]:
train_batch_count = ( train_len - SEQUENCE_LEN) // BATCH_SIZE
test_batch_count = ( test_len - SEQUENCE_LEN) // BATCH_SIZE

In [41]:
print(train_batch_count)
print( test_batch_count )

22585
4211


In [42]:
model = build_model( LAYER_COUNT , SEQUENCE_LEN , VOCABULARY_SIZE , DROPOUT)

In [ ]:
filepath = "../data/models/petrobot_weights"
#call backs 
checkpoint = ModelCheckpoint(
    filepath,
    save_weights_only=True
)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

callbacks_list = [ checkpoint , early_stopping ]

In [ ]:
history = model.fit_generator(
    batch_generator( full_train_text , count=train_batch_count ),
    train_batch_count,
    max_queue_size=1, # no more than one queued batch in RAM
    epochs=EPOCHS,
    callbacks=callbacks_list,
    validation_data=batch_generator( full_test_text , count=test_batch_count ),
    validation_steps=test_batch_count,
    initial_epoch=0 , 
    verbose = 1 
)

Epoch 1/20
22585/22585 [==============================] - 24241s 1s/step - loss: 2.0547 - val_loss: 1.6706
Epoch 2/20
22585/22585 [==============================] - 24236s 1s/step - loss: 1.7495 - val_loss: 1.4823
Epoch 3/20
22585/22585 [==============================] - 24236s 1s/step - loss: 1.4853 - val_loss: 1.3718
Epoch 4/20
22585/22585 [==============================] - 24207s 1s/step - loss: 1.4114 - val_loss: 1.3261
Epoch 5/20
22585/22585 [==============================] - 24209s 1s/step - loss: 1.3747 - val_loss: 1.3017
Epoch 6/20
22585/22585 [==============================] - 24406s 1s/step - loss: 1.3532 - val_loss: 1.2865
Epoch 7/20
22585/22585 [==============================] - 24612s 1s/step - loss: 1.3375 - val_loss: 1.2752
Epoch 8/20
22585/22585 [==============================] - 24140s 1s/step - loss: 1.3262 - val_loss: 1.2644
Epoch 9/20
22585/22585 [==============================] - 24124s 1s/step - loss: 1.3175 - val_loss: 1.2624
Epoch 10/20
22585/22585 [============